# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

The following code builds a single-parameter variant of the [thermal block benchmark from MOR Wiki](https://morwiki.mpi-magdeburg.mpg.de/morwiki/index.php/Thermal_Block#Single_parameter) of the form
$$
\begin{align*}
  E \dot{x}(t, \mu) & = (A_0 + \mu A_1) x(t, \mu) + B u(t), \\
  y(t, \mu) & = C x(t, \mu).
\end{align*}
$$
The parameter domain is $[10^{-6}, 10^2]$.

Since $A$ is parametric, the `LTIModel.from_matrices` method cannot be used and the pyMOR operators need to be created before using the `LTIModel` constructor. The non-parametric operators can be `NumpyMatrixOperators`, while $A$ can be represented using a `LincombOperator` of non-parametric operators and with one scalar being a `ParameterFunctional`.

In [ ]:
from pymor.models.iosys import LTIModel
from pymor.operators.constructions import LincombOperator
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [ ]:
mat = spio.loadmat('../lectures/data/ABCE.mat')
mat.keys()

In [ ]:
A0 = NumpyMatrixOperator(mat['A0'])
A1 = NumpyMatrixOperator(sum(0.2 * i * mat[f'A{i}'] for i in range(1, 5)))
A = LincombOperator((A0, A1), (1, ProjectionParameterFunctional('mu', index=0)))
B = NumpyMatrixOperator(mat['B'])
C = NumpyMatrixOperator(mat['C'])
E = NumpyMatrixOperator(mat['E'])

In [ ]:
fom = LTIModel(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Parametric magnitude plot

We can plot the surface of $\| H(i \omega, \mu) \|$ to see the dynamic behavior of the model.

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf = np.array([[spla.norm(...)  # replace "..." with code to evaluate H(i * w, mu)
                for w in w_list]
               for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf, shading='gouraud', norm=colors.LogNorm())
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Frequency $\omega$')
ax.set_ylabel(r'Parameter $\mu$')
ax.set_title(r'$\Vert H(i \omega, \mu) \Vert$')
_ = fig.colorbar(pcm)

# Hankel singular values

In [ ]:
mu_list_hsv = np.logspace(-6, 2, 3)
hsv_list = [... for mu in mu_list_hsv]  # write code to get Hankel singular values for parameter mu

In [ ]:
fig, ax = plt.subplots()
for mu, hsv in zip(mu_list_hsv, hsv_list):
    ax.semilogy(hsv, '.-', label=fr'$\mu$ = {mu}')
ax.legend()
_ = ax.set_title('Hankel singular values')

# Balanced truncation

Reduce the model for $\mu = 0.01$ using balanced truncation to order $10$.

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(...)

In [ ]:
rom0 = bt.reduce(...)

Note that the ROM is not parametric.

In [ ]:
rom0

`LTIPGReductor` can be used to obtain a parametric ROM.

In [ ]:
from pymor.reductors.basic import LTIPGReductor

In [ ]:
pg = LTIPGReductor(fom, ...)  # use bt.V and bt.W

In [ ]:
rom_pg = pg.reduce()

The parametric structure should be preserved.

In [ ]:
rom_pg

As above, we can plot the err_pgor $\| H(i \omega, \mu) - \hat{H}(i \omega, \mu) \|$

In [ ]:
err_pg = fom - rom_pg

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf_err_pg = np.array([[spla.norm(...)
                    for w in w_list]
                   for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf_err_pg, shading='gouraud', norm=colors.LogNorm())
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Frequency $\omega$')
ax.set_ylabel(r'Parameter $\mu$')
ax.set_title(r'$\Vert H(i \omega, \mu) - \hat{H}(i \omega, \mu) \Vert$')
_ = fig.colorbar(pcm)

The ROM looks accurate around $\mu = 0.01$, but not close to parameter domain boundaries. Check if the ROM is asymptotically stable for $\mu \in \{10^{-6}, 10^{-2}, 10^{2}\}$.

# Building global bases

A simple approach to improving accuracy over the parameter domain is concatenating bases obtained from running balanced truncation for a few different parameter values.

First, initilize empty `VectorArrays` `V` and `W` using the `empty` method of the `fom.A.source` `VectorSpace`.

In [ ]:
V = ...
W = ...

Next, append local bases from balanced truncation

In [ ]:
mu_list_bt = np.logspace(-6, 2, 3)
for mu in mu_list_bt:
    bt_mu = ...
    rom_bt_mu = bt.reduce(10)
    V.append(...)
    W.append(...)

Use POD on the bases to find dominant directions (and orthonormalize the bases).

In [ ]:
from pymor.algorithms.pod import pod

In [ ]:
... = pod(V)
... = pod(W)

Optionally, truncate the bases from POD, and then use them to project the FOM.

In [ ]:
pg_VW = LTIPGReductor(fom, ...)

In [ ]:
rom_pg_VW = pg_VW.reduce()

Plot the error as above and check asymptotic stability.

# Galerkin projection

Since $E$ and $A(\mu)$ in the FOM are respectively positive and negative definite, the same will be true for $V^{\operatorname{T}} E V$ and $V^{\operatorname{T}} A(\mu) V$, for any $V$ of full column rank. Therefore, we can use Galerkin projection to guarantee the preservation of asymptotic stability.

Concatenate `V` and `W` into a single `VectorArray`.

In [ ]:
from pymor.vectorarrays.constructions import cat_arrays

Run POD on the new basis.

Truncate the result from POD (optional) and project the FOm using it.

Plot the error and check asymptotic stability as before.